# Text Preprocessing

In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
data = pd.read_csv('Resources/spam.csv', encoding='latin-1')
data.head()

,label,msg,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
list = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
data = data.drop(list, axis=1)

In [4]:
from gensim.models import KeyedVectors

# loading the downloaded model
w2v = KeyedVectors.load_word2vec_format(
    'Resources/GoogleNews-vectors-negative300.bin', binary=True)

C:\Users\manoh\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Below Function going to break after fullstop words

In [5]:
def breakdot(word):
    tem = []
    fa = 0
    for i in range(len(word)):
        if word[i] == '.':
            fa = fa + 1
        if fa == 2:
            break
    if fa < 2:
        tem = word.split('.')
        return tem
    else:
        tem.append(word)
        return tem
        

In below block data broken into words and got vector from each word. Ignoring unknown words.

In [101]:
data2vec = []
for row in data.itertuples():
    temp = np.ndarray(shape=(1, 300), dtype='float')
    #t = 0
    words = row[2].lower()
    words = words.split()
    for word in words:
        te = breakdot(word)
        for q in te:
            try:
                #if t <= 30:
                mod = w2v[q].reshape(1, 300)
                temp = np.concatenate((temp, mod), axis=0)
                #temp.append(mod.reshape(1, 300))
                t = t + 1
            except Exception:
                pass
        """while t <= 30:
            mod = np.zeros((1, 300))
            temp = np.concatenate((temp, mod), axis=0)
            #temp.append(mod)
            t = t + 1"""
    temp = np.delete(temp,0,0)
    data2vec.append(temp)

Split the data into training and testing

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    data2vec, data['label'], test_size=0.30, random_state=42)

# getting dummies
Y_train = pd.get_dummies(Y_train).values

In [103]:
X_train = np.array(X_train)
X_train.shape

(3900, 31, 300)

# Desigining classifier

In [104]:
# classifier
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input


I = Input(shape=(None, 300))

model = Sequential()
model.add(LSTM(12, dropout_U=0.2, dropout_W=0.2,
               input_shape=(None,300)))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


C:\Users\manoh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(12, input_shape=(None, 300..., dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.


In [105]:
batch_size = 20
model.fit(X_train, Y_train, nb_epoch = 12, batch_size = batch_size, verbose = 2)

C:\Users\manoh\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/12
 - 18s - loss: 0.4502 - acc: 0.8633
Epoch 2/12
 - 14s - loss: 0.3839 - acc: 0.8641
Epoch 3/12
 - 14s - loss: 0.3677 - acc: 0.8692
Epoch 4/12
 - 14s - loss: 0.3554 - acc: 0.8669
Epoch 5/12
 - 13s - loss: 0.3462 - acc: 0.8715
Epoch 6/12
 - 14s - loss: 0.3412 - acc: 0.8731
Epoch 7/12
 - 15s - loss: 0.3403 - acc: 0.8690
Epoch 8/12
 - 16s - loss: 0.3342 - acc: 0.8690
Epoch 9/12
 - 14s - loss: 0.3263 - acc: 0.8777
Epoch 10/12
 - 15s - loss: 0.3273 - acc: 0.8738
Epoch 11/12
 - 14s - loss: 0.3272 - acc: 0.8800
Epoch 12/12
 - 15s - loss: 0.3300 - acc: 0.8659


In [106]:
X_test = np.array(X_test)
Y_te = model.predict(X_test)

In [107]:
Y_pred = []
for i in range(Y_te.shape[0]):
    if Y_te[i][0] >= Y_te[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')

In [108]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

[[1369   84]
 [ 149   70]]
